<a href="https://colab.research.google.com/github/restartus/covid-projection/blob/rich-dev/Restart_NYC_2020_06_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---

_Proprietary and Confidential. Do not distribute without permission._

---

# Restart Partners  

---
_To:_ Jun Amora (Mayors Office, City of New York)  
_From:_ Bharat Shyam, Rich Tong (Restart)  
_Re:_ Analysis for NYC PPE needs  
_Date:_ 20 May 2020  

--- 

New York City needs a 90-day stockpile for the heathcare workers, first responders and congregate care facilities is really important, but coming up with an estimate for this is difficult given the variability of the infection and the uncertainty in the degree of economic recovery and social mobility. Therefore, we are providing another resource model to augment yours that shows that our figures are within 30%-50% of your bottoms estimate. Given that we are happy to:

- _Refine healthcare estimates_. All models are heavily dependent on estimates of population involved and usage data. 
- _Non-healthcare estimates_. For instance, this model does project needs outside of the healthcare area such as small business, vertical industries and vulnerable populations.
- _Long-term modeling_. We are extending the model to include test equipment, disinfectant wipes and liquid disinfectants, so happy to add things that you need. Also we will be integrating epi and economic models too and would love to partner with you on that.

Given the uncertainties involved, this might help you make the right estimates. What follows next are:

1. Disclaimer. This is not a definitive estimate. You should use other sources and information to make your decisions.
2. Data Sources. We have included the model source data, how the model is constructed and then results. Feel free to use this data and modify as appropriate, but it serves as documentation for all the assumptions made.
3. Model. The way the calculation is done with assumptions and resulting projections
4. Outputs. The conclusions we can draw from the projection.

## Disclaimer
It must be noted that the Restart Partners ("Restart") Equipment Model (the "Model") is made available for public use free of charge. Determining equipment needs for each jurisdiction, entity or other party (each a "User") is a complex and multifaceted decision process. Restart does not does not have the authority or ability to assign empirical risks levels nor make definitive use decisions for any User. Rather, the Model provides one approach to making recommendations that can help Users make decisions about their potential equipment uses by allowing them to calculate their potential requirements. Users are strongly encouraged to consider other sources of information and expressly disclaim any cause of action they may have against Restart arising from or relating to the Model or its analysis. Implementing the equipment levels projected by the Model will not eliminate the risk of COID-19 cases being linked to activites in an economy or workplace. In this context, it is important to note that this equipment alone will not eliminate the risk of infection. All Users should remain informed about and abide by any decisions made by local public health and government authorities regarding specific mitigation efforts, including equipment in the model, as the situation is dynamic.

# Model Data

Because we do not have New York City specific data, we used various open data sources to fill in the five major assumptions in the model:

1. Usage by Population. This cuts the item usage per person per day. This right now is a series of levels. So we have four levels for civilians and then two levels for healthcare workers.
2. Usage per Patient. This is the way Epidemiological models work. That is, given a number of patients, calculate how much they will need. The model currently uses the [WHO Surge Essential Supplies Forecasting Tool v1.2](https://www.who.int/emergencies/diseases/novel-coronavirus-2019/technical-guidance/covid-19-critical-items) and estimates the entire US population use with 1,000 cases and fast transmission and slow response. So this is a very pessimistic scenario. This makes sense when calculating the surge estimates.

# Strategy for the Model

The next steps are a little complicated, but we are converting the entire model into a series of vectorized operations. 

## Daily Usage of Equipment by Protection levels D[l, n]

So first we need the inputs which are the list of protection levels p x the number of items we are tracking n. So this is an p x n matrix where each entry says for protection level l, we have so many items per capita per day. So the first data list is Daily_usage D = l rows x n columns or D.shape = ( l, n )

In Python speak this is Usage_pd since it is a _P_anda _D_ataframe. Right now this is a test matrix that is constructed in the Jupyter Notebook, but longer term, it should be pulled from an database with the suitable annotations.

## Sub-Population Count vector P[p, 1]

In the original model, we had p sub-populations. In the simplest model, it was just two populations: non-employees of healthcare companies and employees. With SOC and other codes, there are close to a thousands. So the populations are a vector that includes a description and a population count.

So for example P = [ 7400, 435 ] which is just about the right numbers for Seattle and is a [ 1, p ] vector

## Sub-Population Usage of Protection Levels U[p, l]

You can think of this as a one-hot matrix in the most simple form. That is for each subpopulation, what level of protection do you need. 

For example, in the simplest case, if there are six protection levels, then if non-employees get level 1 and healthcare employees get level 6, then the matrix looks like:

     0 1 0 0 0 0 0
     0 0 0 0 0 0 1

While date entry is complicated, this let's you take any given population and give it fractions of protection. For instance, if a healthcare employer typically had 50% of it's workers as office workers at level 2, 25% as customer facing and 10% taking care of non COVID and 15% in direct contact, that vector would look like:

    0 0.5 0.25 0 0 0.10 0.15

This gives the modeler great flexibility with employers or any population

The matrix of usage U looks like p rows and l columns

    Usage_pd.shape = [ p, l ]

## Required Equipment per capita per sub-population R[p, n]

So with this, you can see that with a single operation you can get to the actual equipment levels require per person per day for a given population. Note that there is new Python 3.5 syntax for [matrix multiply](https://docs.python.org/3/whatsnew/3.5.html#whatsnew-pep-465)

    U x D = [ p, l ] x [l, n] = R[p, n]
    # in the new Python 3.5 syntax using ampersand
    # np.dot for matrices but not tensors
    R = U @ D
    R = U.dot(D)

In Python Numpy speak, we are doing a matrix [multiply](https://www.tutorialexample.com/understand-numpy-np-multiply-np-dot-and-operation-a-beginner-guide-numpy-tutorial/)

## Total required equipment for a sub-population T[p, n]

Now that we have the per-capita requirements, we need to do a scalar multiply by row

    R[p, n] x P[p, 1] = T[p, n]
    # Or in python using broadcasting which extends P out n columns
    T = R * P

## Merging populations into fewer buckets with Essential index by population E[e, p]

Many times the subpopulations are going to be too large to understand. For instance when there are 800 job classfied by SOC or where there are 350 employer class by NAICS-6, so for convenience, we define essential levels. You can think of the of this as for each population, where do they fit in where they start. Essential (which has changed since version 1.x) can be thought of as the time period of start. So Essential 0 (like Defcon 1), is the most important and so forth. 

This let's you stage start up, so for the example subpopulations of non-healthcare employed and heathcare employed, it might look like a simple matrix across 6 start periods as or more analytically E is e rows and p columns.

So in the example, it says the first population starts at time 0 and then the second starts in week 6

    1 0
    0 0
    0 0
    0 0
    0 0
    0 0
    0 1

But this system also allows a stageed restart, so for example, if you want have the workers to come back in the next period for healthcare employees and this series could even be generated as a lambda with any arbitrary function, so in this example, population 1 starts 50% in week 0, then slows continues. 

While population 2 doesn't start until week 4 and tails up

    0.5 0
    0.4 0
    0.1 0
    0 0
    0 0.2
    0 0.4
    0 0.6

## Requirements by essential index Re[e, n]

In some sense we are doing compression by this, so we are looking at Essential index e is much less than the number of populations p. Or more succinctly e << p and we can get to E with a transpose

    E[e, p] x T[p, n] = e x p * p x n = R[e, n]
    # In python this looks like
    Re = E @ T

## Then there is a Cost per item per essential row

For simplicity we can assume that each essential level has different costs. In reality, the costs will actually be more complicate and C[p, n] which is much more complicate.d

For each item, what is the cost for N95 is $3 and non-surgical is $0.50
    C[e, n] = [ $3, $0.50 ]

## The easy parts Required Cost per day RC[e, n] and Stockpile S[e, n]

OK that was the hard stuff, with these matrices reduced to essential levels and the equipment needed for each, there are just some simple scalar multiplies to get where Cost is a row vector that is all the costsj which is a element-wide multiplication.

## Then the Stockpile need by essential levels is SE[e,1]

So for each essential you need a different stockpile. Usually more essential needs more levels

## Gross cost for equipment by level and stockpile by essential

So both the cross cost and the stockpile are done by level as a element-wide multiplication.

    Gross cost for the equipment = RC[e, n] = R[e, n] * C[e, n].value
    Stockpile needed for d Days = S[e, n] = R[e, n] * SE[e, 1].value

Obviously you may not want to stock pile for all e Essential levels, so you just select what you want for instance S[0] will give you the stockpile needs for the most essential level 0.

## Handling disinfection, goggle types and reuse

We can handle any arbitrary vector of items, just change the list. Longer term, we will have our own GUID system, but right no rely on unique text strings








# Detail of Model

These are the details of the model. It is a good example of the parameters that you will need to add. Make sure that you have good advice from medical authorities when looking over these parameters

In [ ]:
# Get libraries
import pandas as pd
import numpy as np

In [ ]:
# https://colab.research.google.com/notebooks/forms.ipynb#scrollTo=ZCEBZPwUDGOg
#@title Basic Model Parameters
#@markdown ####Enter Model Description here:

model_name = 'NYC Surge Forecast'  #@param
model_description = 'v1.4 WHO Surge'  #@param {type: "string"}
recurrence_index = 45  #@param {type: "slider", min: 0, max: 100}
recovery_index = 62  #@param {type: "slider", min: 0, max: 100}
revision =   103#@param {type: "number"}
date = '2020-05-20'  #@param {type: "date"}
model_type = "surge"  #@param ['surge', '3-month', '6-month']

#@markdown ---
#@markdown ####Daily Usage of Equipment Per Person
units = "10,000" #@param ["1,000,000", "100,000", "10,000", "1,000"] {allow-input: true}

#@markdown ---

## Daily Usage of Equipment __n__ by Protection levels __l__ is D[l, n]

In [ ]:
# Eventually we will do this from a database import, but for now, let's use
# the data that is normally in the Excel sheet and just recreate 
# https://colab.research.google.com/drive/1Bcx54NQePYt88RWWmODrRA1pxz-2tnNW?authuser=5#scrollTo=1xwe8g08yRbG

# Using PEP https://www.python.org/dev/peps/pep-0008/
# For simplicity do as a dictionary
Item_name = [
              'N95 Surgical Respirator',
              'N95 Mask',
              'ASTM 3 Surgical Mask',
              'ASTM 1-2 Surgical Mask',
              'Non-ASTM Mask'
              'Reusable Cotton Mask'
              'Cotton Mask with Ear Loop',
              'Face Shield',
              'Goggles'
              'Gown',
              'Gloves',
              'Shoe Covers',
              'Test Kits',
              'Disinfectant (30ml)',
              'Disinfectant wipes'
            ]

# For this demo, we will just test with two
Level_name = [ 'WA0', 'WA1', 'WA2', 'WA3', 'WA4', 'WA5', 'WA6']
Item_name = [ 'N95 Surgical', 'non ASTM Mask']
print('Item_names', Item_name)
Daily_usage_matrix = [
                [ 0, 0 ],
                [ 0, 1 ],
                [ 0, 2 ],
                [ 0.1, 3],
                [ 0.2, 4],
                [ 0.3, 6],
                [ 1.18, 0]]

### Daily Usage Matrix verification and conversion to Dataframe


In [ ]:
print('Daily_usage_matrix', Daily_usage_matrix)

Daily_usage_df = pd.DataFrame(Daily_usage_matrix,
                              columns = Item_name,
                              index = Level_name)

# use these counts to check the matrix vector bugs
level_count = Daily_usage_df.shape[0]
item_count = Daily_usage_df.shape[1]
print('usage_pd shape is ', Daily_usage_df.shape,
      'protection level count is ', level_count,
      'item count is ', item_count)

print('Daily_usage_pd', Daily_usage_df)

Daily_N95s_usage = Daily_usage_df['N95 Surgical']
print('Daily N95 Surgical Usage', Daily_N95s_usage)

# https://stackoverflow.com/questions/13187778/convert-pandas-dataframe-to-numpy-array
print('Daily usage value in Dataframe', Daily_usage_df.values)

## Population Data by sub-populations p is P[p, 1]

Start with the simplest assumption, two populations, one that is `WA6` and one that is `WA2` as an example. But we will insert more data later once we decide the data source.

In [ ]:
# This is a dummy test case, later we will use extraction first form a
# spreadsheet and then eventually from a data store that is reliable
# And which has revision control

Population_name = ['Total less healthcare employees', 'Employees of healthcare companies']
Population_data = [7179.6, 735.2 ]

print('Population Data', Population_data)

Population_df = pd.DataFrame(Population_data, index = Population_name, columns = ['Population'])
population_count = Population_df.shape[0]
print('population count p', population_count)
print(Population_df)

# https://note.nkmk.me/en/python-type-isinstance/
print('type of Population_name', type(Population_name))

# Usage of PPE by Sub-population p is U[p, l]

Now we have a vector which are the population usages and we have a list of needs, so we need to do a matrix multiply of population by needs. Each entry is the percentage of a population at a given level.

So in this example, 50% of healthcare workers are level 5 and 50% are at level 6

In [ ]:
# Now we need a matrix which is the pop_type x usage_type and the coefficient is just how much is needed for each
# Do this for simplicity start with a zero matrix, we will actually load the data

Usage_by_population_matrix = np.zeros([population_count, level_count])

Usage_by_population_matrix[0,1] = 1.0
Usage_by_population_matrix[1,6] = Usage_by_population_matrix[1, 5] = 0.5
print('Usage_by_population_matrix', Usage_by_population_matrix)

# https://www.geeksforgeeks.org/different-ways-to-create-pandas-dataframe/
Usage_by_population_df = pd.DataFrame(Usage_by_population_matrix,
                                      index = Population_name,
                                      columns = Level_name)
print(Usage_by_population_df)



# Required Equipment n per capita per sub-population p per capita is R[p, n]

This is the first multiplication where we take the two matrices and multiply them together. So this will give us a matrix. Each row is for the populations and then each column shows the daily usage by population.

In [ ]:
print('Daily_usage_df', Daily_usage_df.shape)
print('Usage_by_population_df', Usage_by_population_df.shape)

# Note with Panda multiply the index of rows and the columns have to match
Required_df = Usage_by_population_df @ Daily_usage_df

print('Required_df', Required_df)

# Total Required equipment for each Population T[p, n]

We are now just going to case the Population count vector across the required per capita to get the total required across all populations. So we need [element-wise multiplication](https://stackoverflow.com/questions/40034993/how-to-get-element-wise-matrix-multiplication-hadamard-product-in-numpy) which is denoted and this works because of casting, so P is duplicated for each column. Th syntax is different in each variant, for [Dataframes](https:/stackoverflow.com/questions/21022865/pandas-elementwise-multiplication-of-two-dataframes_)

    # In Matlab
    R .* P 
    # In Numpy
    R * P
    # In 
    R * P.values

This is a pretty easy calculation, you just need the element-wise multiplication of the actual population numbers against the per-capita needs. Because of t he way broadcasting works, the vector is spread properly

$T[p, n] = R[p, n] * P[p, 1].values$

In [ ]:

print('Required_df shape', Required_df.shape)
print(Required_df)
print('Population_df shape', Population_df.shape)
print(Population_df)

Total_required_df = Required_df * Population_df.values
print(Total_required_df)

# another formulation
Total_items_per_population_df = Required_df * Population_df.values
print('Total items for each subpopulation')
print(Total_items_per_population_df)

# Now convert Population rows into Essential rows with E[e, p]
This changes the labels and let's you assign each Population with an Essentiality index. Eventually, the essentiality will represent a time series. So e=0 means start at week 0 (for healthcare) and then e=N means start at week N. 

We can even do a time series on that too, say have a sigmoid for the starting or some other sort of lambda.


In [ ]:
# this is a square inverse as we healthcare works are week 0 and non-healthcare is week 1

Essential_name = [ "Essential",
                   "Non-essential"]

Essential_by_population_matrix = [
                                    [0, 1],
                                    [1, 0]                                 
                                  ]

Essential_by_population_df = pd.DataFrame(Essential_by_population_matrix,
                                      index = Essential_name,
                                      columns = Population_name)

print('Essential by population', Essential_by_population_df)

# Now use that matrix to convert Required equipment by population to Required by Essentiality

So we that T[p, n] and we convert with another matrix multiple as

    RE[e, n] = E[e, p] @ T[p, n] 

In [ ]:
Required_by_essential_df = Essential_by_population_df @ Total_required_df
print('Require by Essential Index', Required_by_essential_df)

# Cost matrix is the Cost per item for all items C[e, n]

For each row, we get a cost for the item, so for intance, if N95 surgicals are $3 and non-ASTM disposables are $0., the the vector looks like, because of broadcasting, if you just define a single row, it will be copied against all esesential levels

    CE[e, n] =[ $3.00, $0.50]

This should be the same width as the Product slicing.

In this simple case, all costs across all essentials are identical, but in the more sophisticated costs, costs will vary by volume, so cost becomes a different across different essential levels as volumes and purchasing power are different

    CE[e, n] = [ [ $3, $0.20 ],
                [ $5, $0.50 ]
              ]

In [ ]:
# dataframe is a matrix
# Assumes the same price for all users
# Note that to make the math work, it need to a Numpy Array
Cost_per_item_by_essential_matrix = np.array([ 3, 0.5] )

# Assumes a different price depending on the essnetial level is 50% more
Cost_per_item_by_essential_matrix = [ Cost_per_item_by_essential_matrix, Cost_per_item_by_essential_matrix * 1.5 ]
Cost_per_item_by_essential_df = pd.DataFrame(Cost_per_item_by_essential_matrix,
                                index = Essential_name,
                                columns = Item_name)
print('Cost per item',Cost_per_item_by_essential_df)

# Now calculate the costs based on Requirements and Cost matrix

this is just the element-wise multiplication

TE[e, n] = RE[e, n] * CE[e, n].value

In [ ]:
Total_cost_by_essential_df = Required_by_essential_df * Cost_per_item_by_essential_df.values
print('Total cost by essential_df', Total_cost_by_essential_df )

# Finally use the Stockpile in days per essential to get the Stockpile by essential population

This is another element wise multiply

S[e, n] = RE[e, n] * DE[1, n].values

In [ ]:
# how much stockpile per item is needed for each level
Day_stockpile_by_essential_matrix = [ [30], 
                                     [0]]
Day_stockpile_by_essential_df = pd.DataFrame(Day_stockpile_by_essential_matrix,
                                             index= Essential_name)

print('Day_stockpile_by essential', Day_stockpile_by_essential_df,)

# use .to_numpy as clearer than .values but this does not work
Stockpile_by_essential_df = Required_by_essential_df * Day_stockpile_by_essential_df.values

print('Stockpile by essential', Stockpile_by_essential_df)

## 3D Stack by Time using Multi-index Pandas and Numpy Tensors

You can take any matrix and make it time oriented. So let's take the essential Stockpile and vary it by time.

What is easiest to do with the time series feature of is to flatten all the matrices. And then it seems you just need to add a timestamp.

The other apporach is to use the so called multiindex which let's you do multi dimensionals in a 2-D way.

The use of iterables makes it easy to mix the labels in [multiindexing](https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html) but this is really a display thing. You cannot easily take a multidimensional cube and do a multiply. so the plan is to convert the multi-index to a numpy tensor do the math and then put it back.



In [ ]:
# https://stackoverflow.com/questions/25440008/python-pandas-flatten-a-dataframe-to-a-list
Stockpile_by_essential_df.values.flatten()


# Attachment: Test Code

Used to test various features of the notebook

## Test of cloning an external Repo

NOte that this does a complete clone in the virtual machine, make sure you have enough space. Also you need to reclone when you close a Notebook instance, so this can be slow with lots of data.

However, it does allow you checkout particular branches and have a realiable dataset.

In [ ]:
# Clone the entire repo.
!git clone -l -s git://github.com/jakevdp/PythonDataScienceHandbook.git cloned-repo
%cd cloned-repo
!ls

## Test of copying a single file from a repo

This one way to get small datasets, you just point to the raw file and use `!curl` to bring it into the machine.

In [ ]:
# Fetch a single <1MB file using the raw GitHub URL.
!curl --remote-name \
     -H 'Accept: application/vnd.github.v3.raw' \
     --location https://api.github.com/repos/jakevdp/PythonDataScienceHandbook/contents/notebooks/data/california_cities.csv

## Test of connecting to Google Drive

This we can use if we don't need a repo, but are just loading a static file. We normally want everything from a repo or reliable storage, but this is good for quick analysis. In most cases, you should just check this into a repo and then use the github raw extract instead so you get version control.

Note that this does require an authentication everytime you start the Notebook, so the raw extract works better particularly if there it is a public repo.

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
with open('/gdrive/My Drive/foo.txt', 'w') as f:
  f.write('Hello Google Drive!')
!cat '/gdrive/My Drive/foo.txt'

## Connecting two cells together for summaries with Cross-output Communications

This is the best method for connecting the longer analysis to a cell that just has the executive summary data. _This does not appear to be working. Need to debug_

In [ ]:
%%javascript
const listenerChannel = new BroadcastChannel('channel');
listenerChannel.onmessage = (msg) => {
  const div = document.createElement('div');
  div.textContent = msg.data;
  document.body.appendChild(div);
};

In [ ]:
%%javascript
const senderChannel = new BroadcastChannel('channel');
senderChannel.postMessage('Hello world!');

## Creating forms for entry

This is going to be used to parameterize models. This sets global variables that can be used in cells farther down.

In [ ]:
#@title Example form fields
#@markdown Forms support many types of fields.

no_type_checking = ''  #@param
string_type = 'example'  #@param {type: "string"}
slider_value = 142  #@param {type: "slider", min: 100, max: 200}
number = 102  #@param {type: "number"}
date = '2010-11-05'  #@param {type: "date"}
pick_me = "monday"  #@param ['monday', 'tuesday', 'wednesday', 'thursday']
select_or_input = "apples" #@param ["apples", "bananas", "oranges"] {allow-input: true}
#@markdown ---


## Display Pandas data dataframes use Vega datasets as an example

This uses the extension `google.colab.data_table` and there is a default data set called `vega_datasets` where you can extract data. It is not clear where the data is or how to figure out how ot use it. Google-fu does not help although the [source code](https://github.com/googlecolab/colabtools/blob/master/google/colab/data_table.py) tells us that `vega_dataset` has airport data in it.

But the hing is in the name Vega which is a visualization package and [Vega Datasets access from Python](https://github.com/jakevdp/vega_datasets) are a standard set of data for visualization testing. The core datasets are kept in [github.io](https://vega.github.io/vega-datasets/).

In [ ]:
%load_ext google.colab.data_table

In [ ]:
from vega_datasets import data
data.cars()

In [ ]:
%unload_ext google.colab.data_table

In [ ]:
data.stocks()

## Github Rendering of Jupyter

This is pretty cool, but [Github](https://help.github.com/en/github/managing-files-in-a-repository/working-with-jupyter-notebook-files-on-github) actually renders the Jupyter notebooks as statis HTML when you browse it. That means just clicking on a `.ipynb` will give you something reasonable. It is not interactive nor is anything running behind it, but it does mean that documents produced by use are easily readable.